In [31]:
from symspellpy.symspellpy import SymSpell
import pkg_resources

# Set max_dictionary_edit_distance to avoid spelling correction
sym_spell = SymSpell(max_dictionary_edit_distance=0, prefix_length=7)
dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")
# term_index is the column of the term and count_index is the
# column of the term frequency
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

# a sentence without any spaces
input_term = "thequickbrownfoxjumpsoverthelazydog"
result = sym_spell.word_segmentation(input_term)
print("{}, {}, {}".format(result.corrected_string, result.distance_sum,
                          result.log_prob_sum))

the quick brown fox jumps over the lazy dog, 8, -34.491167981910635


In [4]:
from collections import defaultdict

In [5]:
import re

In [12]:
text = "ソフトバンク, 現在の 支援活動 製品の お知らせ TOPICS トピックス HOT WORD MENU メニュー インターネット・固定電話 でんき ロボット 法人のお客さま1,Google Pixel | スマートフォン・携帯電話 | ソフトバンク, Google Pixel Previous Next Google Pixel 4a (5G) Google Pixel 5 Google Pixel 4a Google Pixel Buds マカロニえんぴつの音楽×トークライブイベント YouTube Premium Google のプレミアム会員サービス Google Pixel 一覧 Google Pixel を彩る豊富なアクセサリー お見積もり・ご購入 購入サポート に関するよくあるご質問 （別ウィンドウで開きます） PDF （モーダルで開きます"

In [38]:
d.check('hello')

True

In [49]:
import enchant
checker = enchant.DictWithPWL("en","correction_vocab.txt")


In [11]:
d = enchant.DictWithPWL("en","correction_vocab1.txt")

In [19]:
import re

In [12]:
import pandas as pd
df = pd.read_csv('/home/iftekhar/amiebot/Resources/test/data/corpus.csv')

In [17]:
text = ' '.join(df.text.values)

In [24]:
collector = []
for items in text.split():
    if bool(re.compile('.*[A-Za-z].*').match(items)):
        clean = re.sub(r'[^A-Za-z]','', items)
        if len(clean) > 1:
            collector.append(clean)

In [25]:
collector = list(set([x.lower() for x in collector]))

In [39]:
joint_words = []
for items in collector:
    if len(items) > 10:
        result = sym_spell.word_segmentation(items)
        joint_words.extend(result.corrected_string.split())
    else:
        joint_words.append(items)

In [47]:
collector = list(set([x for x in joint_words if len(x) > 2]))

In [48]:
with open('correction_vocab.txt', 'w') as out:
    out.writelines("%s\n" % vocab for vocab in collector)

In [1]:
words = ['t', 'HIS', 'web', 'SIT', 'e', 'u', 'SES', 'JavaScript', 'p', 'l', 'ease', 'l', 'e', 't', 'YOU', 'r', 'browser', 's', 'Ett', 'ING', 's', 'Enable', 'JavaScript', 'mobile', 'SEA', 'r', 'c', 'h', 'CLOSE', 'p', 'l', 'ease', 'c', 'h', 'o', 'OSE', 'the', 'SE', 'r', 'vice', 'YOU', 'w', 'ant', 't', 'o', 'k', 'NOW', 'p', 'r', 'eViO', 'us', 'NeXT', 'iPhone SE', 'iPhone', 'iPhone', 'p', 'r', 'o', 'iPad Pro', 'Google Pixel', 'a', 'GET', 't', 'ING', 'a', 'h', 'ANDS', 'e', 't', 'i', 'n', 'Japan', 'ViS', 'i', 't', 'a', 'SoftBank', 'SHO', 'p', 'w', 'e', 'WILL', 'b', 'e', 'HAPPY', 't', 'o', 'h', 'ELP', 'YOU', 'for', 'n', 'e', 'w', 'c', 'o', 'NTR', 'a', 'CTS', 'SWITCH']

In [ ]:
['t', 'HIS', 'web', 'SIT', 'e', 'u', 'SES', 'JavaScript', 'p', 'l', 'ease', 'l', 'e', 't', 'YOU', 'r', 'browser', 's', 'Ett', 'ING', 's', 'Enable', 'JavaScript', 'mobile', 'SEA', 'r', 'c', 'h', 'CLOSE', 'p', 'l', 'ease', 'c', 'h', 'o', 'OSE', 'the', 'SE', 'r', 'vice', 'YOU', 'w', 'ant', 't', 'o', 'k', 'NOW', 'p', 'r', 'eViO', 'us', 'NeXT', 'iPhone SE', 'iPhone', 'iPhone', 'p', 'r', 'o', 'iPad Pro', 'Google Pixel', 'a', 'GET', 't', 'ING', 'a', 'h', 'ANDS', 'e', 't', 'i', 'n', 'Japan', 'ViS', 'i', 't', 'a', 'SoftBank', 'SHO', 'p', 'w', 'e', 'WILL', 'b', 'e', 'HAPPY', 't', 'o', 'h', 'ELP', 'YOU', 'for', 'n', 'e', 'w', 'c', 'o', 'NTR', 'a', 'CTS', 'SWITCH'...

In [3]:
words = [x.lower() for x in words]
correction = defaultdict(list)
i = 0
for items in words:
    flag = 1
    if len(items) < 3 and bool(re.compile('[a-z]').match(items)):
        # print(items, '=', i)
        try:
            j = 0
            while j < 2:
                if checker.check(str(words[i - 1 + j] + words[i + j])):
                    correction[str(words[i - 1 + j] + words[i + j])].append([i - 1 + j, i + j])
                    flag = 0
                    break
                j += 1
            if flag:
                k = 0
                while k < 3:
                    if checker.check(str(words[i - 2 + k] + words[i - 1 + k] + words[i + k])):
                        correction[str(words[i - 2 + k] + words[i - 1 + k] + words[i + k])]. \
                            append([i - 2 + k, i - 1 + k, i + k])
                        flag = 0
                        break
                    k += 1
            if flag:
                k = 0
                while k < 4:
                    if checker.check(str(words[i - 3 + k] + words[i - 2 + k] + words[i - 1 + k] + words[i + k])):
                        correction[str(words[i - 3 + k] + words[i - 2 + k] + words[i - 1 + k] + words[i + k])]. \
                            append([i - 3 + k, i - 2 + k, i - 1 + k, i + k])
                        break
                    k += 1
        except IndexError:
            pass
    i += 1

for key, val in correction.items():
    flag = 1
    for items in val[0]:
        if flag:
            words[items] = key
            flag = 0
        else:
            words[items] = ''

# return [x for x in words if x is not '']


NameError: name 'defaultdict' is not defined

In [53]:
correction

defaultdict(list,
            {'hot': [[8, 9], [8, 9]],
             'menu': [[11, 12]],
             'previous': [[25, 26, 27], [25, 26, 27]],
             'pdf': [[64, 65], [64, 65]]})

In [109]:
d.check('pdf')

True

In [55]:
for key, val in correction.items():
    flag = 1
    for items in val[0]:
        if flag:
            words[items] = key
            flag = 0
        else:
            words[items] = ''

In [57]:
[x for x in words if x is not '']

['ソフトバンク',
 '現在',
 '支援',
 '活動',
 '製品',
 'お知らせ',
 'topics',
 'トピックス',
 'hot',
 'WORD',
 'menu',
 'メニュー',
 'インターネット',
 '固定電話',
 'でんき',
 'ロボット',
 '法人',
 'お客様',
 'Google Pixel',
 'smartphone',
 '携帯電話',
 'ソフトバンク',
 'Google Pixel',
 'previous',
 'NeXT',
 'Google Pixel',
 'a',
 'g',
 'Google Pixel',
 'Google Pixel',
 'a',
 'Google Pixel',
 'buds',
 'マカロニえんぴつ',
 '音楽',
 'トーク',
 'ライブイベント',
 'YouTube Premium',
 'Google',
 'プレミアム',
 '会員',
 'サービス',
 'Google Pixel',
 '一覧',
 'Google Pixel',
 '彩る',
 '豊富',
 'アクセサリー',
 'お',
 '見積',
 'ご',
 '購入',
 '購入',
 'サポート',
 'ある',
 'ご',
 '質問',
 '別',
 'ウィンドウ',
 '開く',
 'pdf',
 'モーダル',
 '開く']

In [59]:
import os
os.path.join('mecab', 'abc.txt')

'mecab/abc.txt'